# Pandas #2
- 학습목표
    - 정제되지 않은 데이터를 여러 형태로 변환
    - 여러 기능 실습
    - 데이터 전처리 하는 방법
    
    
- 핵심키워드
    - pandas
    - groupby
    - pivot_table
    - Hierarchical index
    - aggregation
    - transformation
    - merge & concat
    - join
    - DB persistence

In [1]:
import pandas as pd
import numpy as np

# Groupby
- SQL groupby 명령어와 같음
- split => apply => combine
- 과정을 거쳐 연산함
- `df.groupby('기준컬럼')['적용받는컬럼'].agg(['적용연산'])`

<center><img src="https://image.slidesharecdn.com/slides-151008060416-lva1-app6892/95/pandas-powerful-data-analysis-tools-for-python-19-638.jpg?cb=1444284343" style="height: 80% width: 80%"></center>

In [2]:
df = pd.DataFrame([
    [0],
    [5],
    [10],
    [5],
    [10],
    [15],
    [10],
    [15],
    [20],
])
df.index = ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C']
print(df)
print()
df.reset_index().groupby(['index'])[0].agg(['sum', 'mean'])

    0
A   0
B   5
C  10
A   5
B  10
C  15
A  10
B  15
C  20



,sum,mean
index,,
A,15,5
B,30,10
C,45,15


In [3]:
df = pd.read_csv('wages.csv')
df.head()

,earn,height,sex,race,ed,age
0,79571.299011,73.89,male,white,16,49
1,96396.988643,66.23,female,white,16,62
2,48710.666947,63.77,female,white,16,33
3,80478.096153,63.22,female,other,16,95
4,82089.345498,63.08,female,white,17,43


In [4]:
df.groupby(['sex', 'race'])['earn'].mean()

sex     race    
female  black       26413.283253
        hispanic    21217.352092
        other       34164.346197
        white       23948.241172
male    black       31778.720282
        hispanic    31818.390677
        other       29189.706266
        white       48951.731450
Name: earn, dtype: float64

## Hierarchical index - unstack()
- Group으로 묶여진 데이터를 matrix 형태로 전환해줌
- Series 자료형에서 사용가능 (multi index)

In [5]:
df_unstack = df.groupby(['sex', 'race'])['earn'].mean()
df_unstack

sex     race    
female  black       26413.283253
        hispanic    21217.352092
        other       34164.346197
        white       23948.241172
male    black       31778.720282
        hispanic    31818.390677
        other       29189.706266
        white       48951.731450
Name: earn, dtype: float64

In [6]:
df_unstack.unstack()

race,black,hispanic,other,white
sex,,,,
female,26413.283253,21217.352092,34164.346197,23948.241172
male,31778.720282,31818.390677,29189.706266,48951.731450


In [7]:
df_unstack.to_excel('test.xlsx', '성별&인종에 따른 수입평균')

## Hierarchical index - swaplevel()
- Multi Index의 순서를 바꿔준다.
- index level을 변경할 수 있음

In [8]:
print( df_unstack )
print()

df_unstack.swaplevel().sort_index()

sex     race    
female  black       26413.283253
        hispanic    21217.352092
        other       34164.346197
        white       23948.241172
male    black       31778.720282
        hispanic    31818.390677
        other       29189.706266
        white       48951.731450
Name: earn, dtype: float64



race      sex   
black     female    26413.283253
          male      31778.720282
hispanic  female    21217.352092
          male      31818.390677
other     female    34164.346197
          male      29189.706266
white     female    23948.241172
          male      48951.731450
Name: earn, dtype: float64

## Hierarchical Index - Operations
- index level을 기준으로 기본 연산 수행 가능

In [9]:
df_unstack.swaplevel().sort_index()

race      sex   
black     female    26413.283253
          male      31778.720282
hispanic  female    21217.352092
          male      31818.390677
other     female    34164.346197
          male      29189.706266
white     female    23948.241172
          male      48951.731450
Name: earn, dtype: float64

In [10]:
df_unstack.swaplevel().sort_index().sum(level=0)

race
black       58192.003536
hispanic    53035.742769
other       63354.052463
white       72899.972622
Name: earn, dtype: float64

In [11]:
df_unstack.swaplevel().sort_index().sum(level=1)

sex
female    105743.222714
male      141738.548676
Name: earn, dtype: float64

# Groupby #2

## gropued
- Groupby에 의해 split된 상태를 추출 가능함

In [13]:
grouped = df.groupby('sex')

In [14]:
for name, group in grouped:
    print(name)
    print(group)

female
               earn  height     sex      race  ed  age
1      96396.988643   66.23  female     white  16   62
2      48710.666947   63.77  female     white  16   33
3      80478.096153   63.22  female     other  16   95
4      82089.345498   63.08  female     white  17   43
5      15313.352901   64.53  female     white  15   30
...             ...     ...     ...       ...  ..  ...
1368    4184.222685   60.19  female  hispanic   6   71
1370  175901.453598   65.90  female     other  18   52
1373   16905.557851   70.08  female     white  16   40
1375   24853.519514   61.31  female     white  18   86
1376   13710.671312   63.64  female     white  12   37

[859 rows x 6 columns]
male
              earn  height   sex      race  ed  age
0     79571.299011   73.89  male     white  16   49
7     50960.054282   73.29  male     white  17   50
8      3212.649556   72.24  male  hispanic  15   25
9     42996.637884   72.40  male     white  12   30
10    10328.618843   70.22  male     white  

In [19]:
# 특정 key값을 가진 그룹의 정보만 추출 가능
# df[df['sex'] == 'male']
grouped.get_group('male')

,earn,height,sex,race,ed,age
0,79571.299011,73.89,male,white,16,49
7,50960.054282,73.29,male,white,17,50
8,3212.649556,72.24,male,hispanic,15,25
9,42996.637884,72.40,male,white,12,30
10,10328.618843,70.22,male,white,16,69
...,...,...,...,...,...,...
1371,87473.968778,68.82,male,white,18,75
1372,92205.596106,69.62,male,white,18,57
1374,30173.380363,71.68,male,white,12,33
1377,95426.014410,71.65,male,white,12,54


In [22]:
# Aggregation: 요약된 통계정보를 추출해 줌
# Transformation: 해당 정보를 변환해줌
# Filtration: 특정 정보를 제거하여 보여주는 필터링 기능
grouped.agg(sum)

,earn,height,ed,age
sex,,,,
female,2.082701e+07,55408.10,11371,39676
male,2.391643e+07,36423.15,7045,22832


In [24]:
grouped.agg(np.mean)

,earn,height,ed,age
sex,,,,
female,24245.647877,64.503027,13.237485,46.188591
male,45993.126922,70.044519,13.548077,43.907692


In [28]:
# filter
df.groupby('sex').filter(lambda x: len(x) >= 6)

,earn,height,sex,race,ed,age
0,79571.299011,73.89,male,white,16,49
1,96396.988643,66.23,female,white,16,62
2,48710.666947,63.77,female,white,16,33
3,80478.096153,63.22,female,other,16,95
4,82089.345498,63.08,female,white,17,43
...,...,...,...,...,...,...
1374,30173.380363,71.68,male,white,12,33
1375,24853.519514,61.31,female,white,18,86
1376,13710.671312,63.64,female,white,12,37
1377,95426.014410,71.65,male,white,12,54


In [29]:
df.shape

(1379, 6)

In [30]:
!wget https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv

--2020-11-02 19:07:59--  https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv
Resolving www.shanelynn.ie (www.shanelynn.ie)... 104.236.88.249
Connecting to www.shanelynn.ie (www.shanelynn.ie)|104.236.88.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40576 (40K) [text/csv]
Saving to: ‘phone_data.csv’

phone_data.csv      100%[===================>]  39.62K   199KB/s    in 0.2s    

2020-11-02 19:08:01 (199 KB/s) - ‘phone_data.csv’ saved [40576/40576]



In [31]:
df_phone = pd.read_csv('phone_data.csv')

In [32]:
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [35]:
import dateutil

df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [38]:
df_phone.iloc[0, 1]

Timestamp('2014-10-15 06:58:00')

In [43]:
# 월별로 어느정도 통화를 썼는지,
df_phone.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [45]:
df_phone[df_phone['item'] == 'call'].groupby('month')['duration'].sum()

month
2014-11    25547.0
2014-12    13561.0
2015-01    17070.0
2015-02    14416.0
2015-03    21727.0
Name: duration, dtype: float64

In [47]:
df_phone.groupby(['month', 'item'])['duration'].sum()

month    item
2014-11  call    25547.000
         data      998.441
         sms        94.000
2014-12  call    13561.000
         data     1032.870
         sms        48.000
2015-01  call    17070.000
         data     1067.299
         sms        86.000
2015-02  call    14416.000
         data     1067.299
         sms        39.000
2015-03  call    21727.000
         data      998.441
         sms        25.000
Name: duration, dtype: float64

In [54]:
df_phone.groupby(['month', 'item'])['date'].count().unstack()

item,call,data,sms
month,,,
2014-11,107,29,94
2014-12,79,30,48
2015-01,88,31,86
2015-02,67,31,39
2015-03,47,29,25


In [57]:
df_phone.groupby(['month', 'item'])['date'].count().unstack().reset_index()

item,month,call,data,sms
0,2014-11,107,29,94
1,2014-12,79,30,48
2,2015-01,88,31,86
3,2015-02,67,31,39
4,2015-03,47,29,25


In [60]:
df_phone.groupby('month', as_index=False).agg({'duration': 'sum'})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


In [69]:
df_phone.groupby('month')['duration'].agg(['sum']).reset_index()

,month,sum
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


## aggregation (agg)
- 제일 많이 쓰인다.

In [71]:
# 제일 좋음,
# 데이터 분석때 가장 많이 쓰인다.

df_phone.groupby(['month', 'item']).agg({
    'duration': sum,
    'network_type': 'count',
    'date': 'first'
})

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

In [79]:
df_phone.groupby(['month', 'item']).agg({
    'duration': sum,
    'network_type': 'count',
    'date': [min, 'first', 'nunique']
})

duration network_type                date                      \
                    sum        count                 min               first   
month   item                                                                   
2014-11 call  25547.000          107 2014-10-15 06:58:00 2014-10-15 06:58:00   
        data    998.441           29 2014-10-15 06:58:00 2014-10-15 06:58:00   
        sms      94.000           94 2014-10-16 22:18:00 2014-10-16 22:18:00   
2014-12 call  13561.000           79 2014-11-14 17:24:00 2014-11-14 17:24:00   
        data   1032.870           30 2014-11-13 06:58:00 2014-11-13 06:58:00   
        sms      48.000           48 2014-11-14 17:28:00 2014-11-14 17:28:00   
2015-01 call  17070.000           88 2014-12-15 20:03:00 2014-12-15 20:03:00   
        data   1067.299           31 2014-12-13 06:58:00 2014-12-13 06:58:00   
        sms      86.000           86 2014-12-15 19:56:00 2014-12-15 19:56:00   
2015-02 call  14416.000           67 2015-01-15 10:36:00 2015-01-15 10:36:00   
        data   1067.299           31 2015-01-13 06:58:00 2015-01-13 06:58:00   
        sms      39.000           39 2015-01-15 12:23:00 2015-01-15 12:23:00   
2015-03 call  21727.000           47 2015-02-12 20:15:00 2015-02-12 20:15:00   
        data    998.441           29 2015-02-13 06:58:00 2015-02-13 06:58:00   
        sms      25.000           25 2015-02-19 18:46:00 2015-02-19 18:46:00   

                      
             nunique  
month   item          
2014-11 call     104  
        data      29  
        sms       79  
2014-12 call      76  
        data      30  
        sms       41  
2015-01 call      84  
        data      31  
        sms       58  
2015-02 call      67  
        data      31  
        sms       27  
2015-03 call      47  
        data      29  
        sms       17

# Pivot Table
- Crosstab
- unstack()

In [81]:
df_phone = pd.read_csv('phone_data.csv')
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [100]:
df_phone.pivot_table(
    ['duration'],
    index = [df_phone['month'], df_phone['item']],
    columns = df_phone['network'],
    aggfunc = 'sum',
    fill_value = 0
)

duration                                                   \
network        Meteor Tesco  Three Vodafone      data landline special   
month   item                                                             
2014-11 call     1521  4045  12458     4316     0.000     2906       0   
        data        0     0      0        0   998.441        0       0   
        sms        10     3     25       55     0.000        0       1   
2014-12 call     2010  1819   6316     1302     0.000     1424       0   
        data        0     0      0        0  1032.870        0       0   
        sms        12     1     13       18     0.000        0       0   
2015-01 call     2207  2904   6445     3626     0.000     1603       0   
        data        0     0      0        0  1067.299        0       0   
        sms        10     3     33       40     0.000        0       0   
2015-02 call     1188  4087   6279     1864     0.000      730       0   
        data        0     0      0        0  1067.299        0       0   
        sms         1     2     11       23     0.000        0       2   
2015-03 call      274   973   4966     3513     0.000    11770       0   
        data        0     0      0        0   998.441        0       0   
        sms         0     4      5       13     0.000        0       0   

                              
network      voicemail world  
month   item                  
2014-11 call       301     0  
        data         0     0  
        sms          0     0  
2014-12 call       690     0  
        data         0     0  
        sms          0     4  
2015-01 call       285     0  
        data         0     0  
        sms          0     0  
2015-02 call       268     0  
        data         0     0  
        sms          0     0  
2015-03 call       231     0  
        data         0     0  
        sms          0     3

In [104]:
len(df_phone['network'].unique())

9

## Crosstab
- 특히, 두 칼럼에 교차 빈도, 비율, 덧셈 등을 구할 때 사용
- Pivot Table의 특수한 형태
- User-item Rating Matrix등을 만들 때 사용가능함

In [106]:
# groupby 와 pivot_table 과 비슷한 기능 수행

# Merge & Concat

## Merge
- 두 개의 데이터프레임을 합칠 때

In [ ]:
# inner join (기본값))

# merge(df1, df2, on='겹치는컬럼명')
pd.merge(df_a, df_b, on='subject_id')

In [ ]:
# 두 개의 컬럼이름이 다를 때,

# left_on => df1, right_on => df2
pd.merge(df1, df2, left_on='subject_id', right_on='subject_id')

---
- Inner Join
- Full Join
- Left Join
- Right Join

<center><img src='https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F9967FA335996B3F12F2CCF'></center>

In [ ]:
# Left Join
pd.merge(df1, df2, on='subject_id', how='left')

In [ ]:
# Right Join
pd.merge(df1, df2, on='subject_id', how='right')

In [ ]:
# Full Join
pd.merge(df1, df2, on='subject_id', how='outer')

## index based join

In [ ]:
pd.merge(df1, df2, left_index=True, right_index=True)

# DB Persistence

## Database connection
- Data loading시 db connection 기능을 제공함

In [ ]:
import sqlite3

conn = sqlite3.connect('./data.db')
cur = conn.cursor()
cur.execute('select * from airlines limit 5;')
results = cur.fetchall()
results

In [ ]:
df_airplines = pd.read_sql_query('SELECT * FROM airlines;', conn)
df_airports = pd.read_sql_query('SELECT * FROM airports;', conn)
df_routes = pd.read_sql_query('SELECT * FROM routes;', conn)

## XLS persistence
- Dataframe 의 엑셀 추출 코드
- Xls 엔진으로 openpyxls 또는 XlsxWrite 사용

In [ ]:
writer = pd.ExcelWriter('./df_routes.xlsx', engine='xlsxwriter')
df_routes.to_excel(writer, sheet_name='Sheet1')